# Web Scraping to complete the dataset

In [27]:
import pandas as pd
import seaborn as sns
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

In [10]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

In [2]:
df_courses = pd.read_csv('../Data/interim/Courses.csv')

In [3]:
df_courses.head()

,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,contentInfo,publishedTime,category,timeSpent,publishDate,level,paidBool
0,28295,Learn Web Designing & HTML5/CSS3 Essentials in...,https://www.udemy.com/build-beautiful-html5-we...,True,75.0,43285,525,24,All Levels,4 hours,2013-01-03T00:55:31Z,WebDevelopment,4.0,2013-01-03,All Levels,True
1,19603,Learning Dynamic Website Design - PHP MySQL an...,https://www.udemy.com/learning-dynamic-website...,True,50.0,47886,285,125,All Levels,12.5 hours,2012-06-18T16:52:34Z,WebDevelopment,12.5,2012-06-18,All Levels,True
2,889438,ChatBots: Messenger ChatBot with API.AI and No...,https://www.udemy.com/chatbots/,True,50.0,2577,529,64,All Levels,4.5 hours,2016-06-30T16:57:08Z,WebDevelopment,4.5,2016-06-30,All Levels,True
3,197836,Projects in HTML5,https://www.udemy.com/projects-in-html5/,True,60.0,8777,206,75,Intermediate Level,15.5 hours,2014-06-17T05:43:50Z,WebDevelopment,15.5,2014-06-17,Intermediate Level,True
4,505208,Programming Foundations: HTML5 + CSS3 for Entr...,https://www.udemy.com/html-css-more/,True,20.0,23764,490,58,Beginner Level,5.5 hours,2015-10-17T04:52:25Z,WebDevelopment,5.5,2015-10-17,Beginner Level,True


In [29]:
headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
        }

In [30]:
req = Request(df_courses.url[0], headers=headers) 
webpage = urlopen(req).read()

In [31]:
page_soup = soup(webpage, 'html.parser')

Extract:
- Requirements </div>
<div class="requirements__content">
<ul class="requirements__list">
<li class="requirements__item">Adobe Photoshop CS3 or higher</li>
<li class="requirements__item">Very basic knowledge of HTML &amp; CSS</li>
<li class="requirements__item">Text Editor (TextWranger, Espresso, or Coda recommended)</li>

In [32]:
page_soup


<!DOCTYPE html>

<html lang="en-us">
<head>
<title>Learn Web Designing &amp; HTML5/CSS3 Essentials in 4-Hours | Udemy</title>
<meta content="Learn how to take a PSD and turn it into a beautiful, hand-coded HTML5 &amp; CSS3 website with this web design crash course." name="description"/><meta content="summary_large_image" name="twitter:card"/><meta content="mult" name="medium"/><meta content="app-id=562413829, affiliate-data=ct=Safari_SmartBanner&amp;amp;pt=1240482" name="apple-itunes-app"/><meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/><meta content="app-id=com.udemy.android" name="google-play-app"/><meta content="Learn Web Designing &amp; HTML5/CSS3 Essentials in 4-Hours" name="title"/><meta content="313137469260" property="fb:app_id"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="1